# Assignment 1

In [6]:
# First, install spacy, pandas, and the language model in the terminal, by typing in:
# pip install spacy pandas
# python -m spacy download en_core_web_md

In [3]:
import spacy
import pandas as pd
import os

In [4]:
nlp = spacy.load("en_core_web_md")

In [5]:
# create a path to the corpus containing our data and then sort the directory
main_folder_path = "../input/USEcorpus/"
sorted_dir = sorted(os.listdir(main_folder_path))

In [6]:
temp_sorted_dir = sorted_dir[:2]

In [7]:
temp_sorted_dir

['a1', 'a2']

In [ ]:
# preprocessing: removing all tokens that are simply document metadata or identificatio

In [ ]:
# creating a for loop going into each subfolder in the corpus and opening all text files within the subfolders
for folder in temp_sorted_dir:
    folder_path = os.path.join(main_folder_path, folder)
    filenames = sorted(os.listdir(folder_path))
    folder_info = []
    
    for text_file in filenames:
        file_path = folder_path + "/" + text_file
        
        with open (file_path, encoding="latin-1") as file:
            text = file.read()
            # add text files to a doc object
            doc = nlp(text)
            # create counters for nouns, verbs, adjectives, and adverbs.
            noun_count = 0
            verb_count = 0
            adj_count = 0
            adv_count = 0

            # with a for loop, add 1 to the counter every time that part of speech appears in the doc object
            for token in doc:
                if token.pos_ == "NOUN":
                    noun_count += 1
                elif token.pos_ == "VERB":
                    verb_count += 1
                elif token.pos_ == "ADJ":
                    adj_count += 1
                elif token.pos_ == "ADV":
                    adv_count += 1
            
            # relative frequencies of nouns, verbs, adjectives, and adverbs per 10,000 words        
            relative_freq_noun = (noun_count/len(doc)) * 10000
            relative_freq_verb = (verb_count/len(doc)) * 10000
            relative_freq_adj = (adj_count/len(doc)) * 10000
            relative_freq_adv = (adv_count/len(doc)) * 10000
            
            # extracting named entities for persons, locations, and organizations
            persons = set()
            for ent in doc.ents:
                    if ent.label_ == 'PERSON':
                        persons.add(ent.text)
            num_persons = len(persons)
            
            locations = set()
            for ent in doc.ents:
                    if ent.label_ == 'LOC':
                        locations.add(ent.text)
            num_locations = len(locations)
            
            organisations = set()
            for ent in doc.ents:
                    if ent.label_ == 'ORG':
                        organisations.add(ent.text)
            num_organisations = len(organisations)
            
            file_info = [text_file, noun_count]
            
            folder_info.append(file_info)
            
            print(text_file, noun_count, verb_count, adj_count, adv_count, num_persons, num_locations, num_organisations)
        
        # concatenate all dataframes saved in list for folder 
        print(folder_info)
        df = pd.DataFrame(folder_info)
        
        df.to_csv("test.csv")
        

0100.a1.txt 111 81 57 42 1 0 0
[['0100.a1.txt', 111]]
0101.a1.txt 93 103 51 72 1 0 1
[['0100.a1.txt', 111], ['0101.a1.txt', 93]]
0102.a1.txt 126 100 58 50 0 0 1
[['0100.a1.txt', 111], ['0101.a1.txt', 93], ['0102.a1.txt', 126]]
0103.a1.txt 97 122 51 60 0 0 1
[['0100.a1.txt', 111], ['0101.a1.txt', 93], ['0102.a1.txt', 126], ['0103.a1.txt', 97]]
0104.a1.txt 106 98 49 65 0 1 2
[['0100.a1.txt', 111], ['0101.a1.txt', 93], ['0102.a1.txt', 126], ['0103.a1.txt', 97], ['0104.a1.txt', 106]]
0105.a1.txt 106 109 48 60 3 0 0
[['0100.a1.txt', 111], ['0101.a1.txt', 93], ['0102.a1.txt', 126], ['0103.a1.txt', 97], ['0104.a1.txt', 106], ['0105.a1.txt', 106]]
0106.a1.txt 127 100 68 77 0 0 0
[['0100.a1.txt', 111], ['0101.a1.txt', 93], ['0102.a1.txt', 126], ['0103.a1.txt', 97], ['0104.a1.txt', 106], ['0105.a1.txt', 106], ['0106.a1.txt', 127]]
0107.a1.txt 88 97 64 55 0 0 0
[['0100.a1.txt', 111], ['0101.a1.txt', 93], ['0102.a1.txt', 126], ['0103.a1.txt', 97], ['0104.a1.txt', 106], ['0105.a1.txt', 106], ['0106

0130.a1.txt 111 92 66 61 1 0 1
[['0100.a1.txt', 111], ['0101.a1.txt', 93], ['0102.a1.txt', 126], ['0103.a1.txt', 97], ['0104.a1.txt', 106], ['0105.a1.txt', 106], ['0106.a1.txt', 127], ['0107.a1.txt', 88], ['0108.a1.txt', 93], ['0109.a1.txt', 112], ['0110.a1.txt', 105], ['0111.a1.txt', 113], ['0112.a1.txt', 110], ['0113.a1.txt', 179], ['0114.a1.txt', 94], ['0115.a1.txt', 109], ['0116.a1.txt', 89], ['0118.a1.txt', 97], ['0119.a1.txt', 101], ['0120.a1.txt', 99], ['0121.a1.txt', 141], ['0122.a1.txt', 125], ['0123.a1.txt', 100], ['0124.a1.txt', 154], ['0125.a1.txt', 154], ['0127.a1.txt', 84], ['0128.a1.txt', 114], ['0130.a1.txt', 111]]
0131.a1.txt 107 106 57 46 0 0 0
[['0100.a1.txt', 111], ['0101.a1.txt', 93], ['0102.a1.txt', 126], ['0103.a1.txt', 97], ['0104.a1.txt', 106], ['0105.a1.txt', 106], ['0106.a1.txt', 127], ['0107.a1.txt', 88], ['0108.a1.txt', 93], ['0109.a1.txt', 112], ['0110.a1.txt', 105], ['0111.a1.txt', 113], ['0112.a1.txt', 110], ['0113.a1.txt', 179], ['0114.a1.txt', 94], ['0

0144.a1.txt 86 97 55 67 0 0 3
[['0100.a1.txt', 111], ['0101.a1.txt', 93], ['0102.a1.txt', 126], ['0103.a1.txt', 97], ['0104.a1.txt', 106], ['0105.a1.txt', 106], ['0106.a1.txt', 127], ['0107.a1.txt', 88], ['0108.a1.txt', 93], ['0109.a1.txt', 112], ['0110.a1.txt', 105], ['0111.a1.txt', 113], ['0112.a1.txt', 110], ['0113.a1.txt', 179], ['0114.a1.txt', 94], ['0115.a1.txt', 109], ['0116.a1.txt', 89], ['0118.a1.txt', 97], ['0119.a1.txt', 101], ['0120.a1.txt', 99], ['0121.a1.txt', 141], ['0122.a1.txt', 125], ['0123.a1.txt', 100], ['0124.a1.txt', 154], ['0125.a1.txt', 154], ['0127.a1.txt', 84], ['0128.a1.txt', 114], ['0130.a1.txt', 111], ['0131.a1.txt', 107], ['0133.a1.txt', 92], ['0134.a1.txt', 111], ['0135.a1.txt', 111], ['0136.a1.txt', 127], ['0137.a1.txt', 90], ['0138.a1.txt', 112], ['0139.a1.txt', 87], ['0140.a1.txt', 146], ['0142.a1.txt', 115], ['0143.a1.txt', 120], ['0144.a1.txt', 86]]
0145.a1.txt 137 100 58 68 0 0 0
[['0100.a1.txt', 111], ['0101.a1.txt', 93], ['0102.a1.txt', 126], ['01

0158.a1.txt 95 121 54 67 0 0 0
[['0100.a1.txt', 111], ['0101.a1.txt', 93], ['0102.a1.txt', 126], ['0103.a1.txt', 97], ['0104.a1.txt', 106], ['0105.a1.txt', 106], ['0106.a1.txt', 127], ['0107.a1.txt', 88], ['0108.a1.txt', 93], ['0109.a1.txt', 112], ['0110.a1.txt', 105], ['0111.a1.txt', 113], ['0112.a1.txt', 110], ['0113.a1.txt', 179], ['0114.a1.txt', 94], ['0115.a1.txt', 109], ['0116.a1.txt', 89], ['0118.a1.txt', 97], ['0119.a1.txt', 101], ['0120.a1.txt', 99], ['0121.a1.txt', 141], ['0122.a1.txt', 125], ['0123.a1.txt', 100], ['0124.a1.txt', 154], ['0125.a1.txt', 154], ['0127.a1.txt', 84], ['0128.a1.txt', 114], ['0130.a1.txt', 111], ['0131.a1.txt', 107], ['0133.a1.txt', 92], ['0134.a1.txt', 111], ['0135.a1.txt', 111], ['0136.a1.txt', 127], ['0137.a1.txt', 90], ['0138.a1.txt', 112], ['0139.a1.txt', 87], ['0140.a1.txt', 146], ['0142.a1.txt', 115], ['0143.a1.txt', 120], ['0144.a1.txt', 86], ['0145.a1.txt', 137], ['0146.a1.txt', 91], ['0148.a1.txt', 105], ['0149.a1.txt', 127], ['0151.a1.txt'

0168.a1.txt 111 112 49 79 1 0 0
[['0100.a1.txt', 111], ['0101.a1.txt', 93], ['0102.a1.txt', 126], ['0103.a1.txt', 97], ['0104.a1.txt', 106], ['0105.a1.txt', 106], ['0106.a1.txt', 127], ['0107.a1.txt', 88], ['0108.a1.txt', 93], ['0109.a1.txt', 112], ['0110.a1.txt', 105], ['0111.a1.txt', 113], ['0112.a1.txt', 110], ['0113.a1.txt', 179], ['0114.a1.txt', 94], ['0115.a1.txt', 109], ['0116.a1.txt', 89], ['0118.a1.txt', 97], ['0119.a1.txt', 101], ['0120.a1.txt', 99], ['0121.a1.txt', 141], ['0122.a1.txt', 125], ['0123.a1.txt', 100], ['0124.a1.txt', 154], ['0125.a1.txt', 154], ['0127.a1.txt', 84], ['0128.a1.txt', 114], ['0130.a1.txt', 111], ['0131.a1.txt', 107], ['0133.a1.txt', 92], ['0134.a1.txt', 111], ['0135.a1.txt', 111], ['0136.a1.txt', 127], ['0137.a1.txt', 90], ['0138.a1.txt', 112], ['0139.a1.txt', 87], ['0140.a1.txt', 146], ['0142.a1.txt', 115], ['0143.a1.txt', 120], ['0144.a1.txt', 86], ['0145.a1.txt', 137], ['0146.a1.txt', 91], ['0148.a1.txt', 105], ['0149.a1.txt', 127], ['0151.a1.txt

...

3068.a1.txt 65 58 35 34 0 0 0
[['0100.a1.txt', 111], ['0101.a1.txt', 93], ['0102.a1.txt', 126], ['0103.a1.txt', 97], ['0104.a1.txt', 106], ['0105.a1.txt', 106], ['0106.a1.txt', 127], ['0107.a1.txt', 88], ['0108.a1.txt', 93], ['0109.a1.txt', 112], ['0110.a1.txt', 105], ['0111.a1.txt', 113], ['0112.a1.txt', 110], ['0113.a1.txt', 179], ['0114.a1.txt', 94], ['0115.a1.txt', 109], ['0116.a1.txt', 89], ['0118.a1.txt', 97], ['0119.a1.txt', 101], ['0120.a1.txt', 99], ['0121.a1.txt', 141], ['0122.a1.txt', 125], ['0123.a1.txt', 100], ['0124.a1.txt', 154], ['0125.a1.txt', 154], ['0127.a1.txt', 84], ['0128.a1.txt', 114], ['0130.a1.txt', 111], ['0131.a1.txt', 107], ['0133.a1.txt', 92], ['0134.a1.txt', 111], ['0135.a1.txt', 111], ['0136.a1.txt', 127], ['0137.a1.txt', 90], ['0138.a1.txt', 112], ['0139.a1.txt', 87], ['0140.a1.txt', 146], ['0142.a1.txt', 115], ['0143.a1.txt', 120], ['0144.a1.txt', 86], ['0145.a1.txt', 137], ['0146.a1.txt', 91], ['0148.a1.txt', 105], ['0149.a1.txt', 127], ['0151.a1.txt',

4013.a1.txt 68 72 36 41 0 0 0
[['0100.a1.txt', 111], ['0101.a1.txt', 93], ['0102.a1.txt', 126], ['0103.a1.txt', 97], ['0104.a1.txt', 106], ['0105.a1.txt', 106], ['0106.a1.txt', 127], ['0107.a1.txt', 88], ['0108.a1.txt', 93], ['0109.a1.txt', 112], ['0110.a1.txt', 105], ['0111.a1.txt', 113], ['0112.a1.txt', 110], ['0113.a1.txt', 179], ['0114.a1.txt', 94], ['0115.a1.txt', 109], ['0116.a1.txt', 89], ['0118.a1.txt', 97], ['0119.a1.txt', 101], ['0120.a1.txt', 99], ['0121.a1.txt', 141], ['0122.a1.txt', 125], ['0123.a1.txt', 100], ['0124.a1.txt', 154], ['0125.a1.txt', 154], ['0127.a1.txt', 84], ['0128.a1.txt', 114], ['0130.a1.txt', 111], ['0131.a1.txt', 107], ['0133.a1.txt', 92], ['0134.a1.txt', 111], ['0135.a1.txt', 111], ['0136.a1.txt', 127], ['0137.a1.txt', 90], ['0138.a1.txt', 112], ['0139.a1.txt', 87], ['0140.a1.txt', 146], ['0142.a1.txt', 115], ['0143.a1.txt', 120], ['0144.a1.txt', 86], ['0145.a1.txt', 137], ['0146.a1.txt', 91], ['0148.a1.txt', 105], ['0149.a1.txt', 127], ['0151.a1.txt',

4043.a1.txt 65 68 33 45 0 0 0
[['0100.a1.txt', 111], ['0101.a1.txt', 93], ['0102.a1.txt', 126], ['0103.a1.txt', 97], ['0104.a1.txt', 106], ['0105.a1.txt', 106], ['0106.a1.txt', 127], ['0107.a1.txt', 88], ['0108.a1.txt', 93], ['0109.a1.txt', 112], ['0110.a1.txt', 105], ['0111.a1.txt', 113], ['0112.a1.txt', 110], ['0113.a1.txt', 179], ['0114.a1.txt', 94], ['0115.a1.txt', 109], ['0116.a1.txt', 89], ['0118.a1.txt', 97], ['0119.a1.txt', 101], ['0120.a1.txt', 99], ['0121.a1.txt', 141], ['0122.a1.txt', 125], ['0123.a1.txt', 100], ['0124.a1.txt', 154], ['0125.a1.txt', 154], ['0127.a1.txt', 84], ['0128.a1.txt', 114], ['0130.a1.txt', 111], ['0131.a1.txt', 107], ['0133.a1.txt', 92], ['0134.a1.txt', 111], ['0135.a1.txt', 111], ['0136.a1.txt', 127], ['0137.a1.txt', 90], ['0138.a1.txt', 112], ['0139.a1.txt', 87], ['0140.a1.txt', 146], ['0142.a1.txt', 115], ['0143.a1.txt', 120], ['0144.a1.txt', 86], ['0145.a1.txt', 137], ['0146.a1.txt', 91], ['0148.a1.txt', 105], ['0149.a1.txt', 127], ['0151.a1.txt',

5015.a1.txt 52 49 24 26 0 0 0
[['0100.a1.txt', 111], ['0101.a1.txt', 93], ['0102.a1.txt', 126], ['0103.a1.txt', 97], ['0104.a1.txt', 106], ['0105.a1.txt', 106], ['0106.a1.txt', 127], ['0107.a1.txt', 88], ['0108.a1.txt', 93], ['0109.a1.txt', 112], ['0110.a1.txt', 105], ['0111.a1.txt', 113], ['0112.a1.txt', 110], ['0113.a1.txt', 179], ['0114.a1.txt', 94], ['0115.a1.txt', 109], ['0116.a1.txt', 89], ['0118.a1.txt', 97], ['0119.a1.txt', 101], ['0120.a1.txt', 99], ['0121.a1.txt', 141], ['0122.a1.txt', 125], ['0123.a1.txt', 100], ['0124.a1.txt', 154], ['0125.a1.txt', 154], ['0127.a1.txt', 84], ['0128.a1.txt', 114], ['0130.a1.txt', 111], ['0131.a1.txt', 107], ['0133.a1.txt', 92], ['0134.a1.txt', 111], ['0135.a1.txt', 111], ['0136.a1.txt', 127], ['0137.a1.txt', 90], ['0138.a1.txt', 112], ['0139.a1.txt', 87], ['0140.a1.txt', 146], ['0142.a1.txt', 115], ['0143.a1.txt', 120], ['0144.a1.txt', 86], ['0145.a1.txt', 137], ['0146.a1.txt', 91], ['0148.a1.txt', 105], ['0149.a1.txt', 127], ['0151.a1.txt',

0100.a2.txt 171 95 68 32 0 0 0
[['0100.a2.txt', 171]]
0101.a2.txt 167 93 45 59 0 0 0
[['0100.a2.txt', 171], ['0101.a2.txt', 167]]
0102.a2.txt 153 94 56 63 1 1 0
[['0100.a2.txt', 171], ['0101.a2.txt', 167], ['0102.a2.txt', 153]]
0103.a2.txt 163 105 86 34 0 0 1
[['0100.a2.txt', 171], ['0101.a2.txt', 167], ['0102.a2.txt', 153], ['0103.a2.txt', 163]]
0104.a2.txt 168 86 57 31 4 0 0
[['0100.a2.txt', 171], ['0101.a2.txt', 167], ['0102.a2.txt', 153], ['0103.a2.txt', 163], ['0104.a2.txt', 168]]
0105.a2.txt 135 96 73 53 0 0 0
[['0100.a2.txt', 171], ['0101.a2.txt', 167], ['0102.a2.txt', 153], ['0103.a2.txt', 163], ['0104.a2.txt', 168], ['0105.a2.txt', 135]]
0106.a2.txt 145 81 62 60 0 0 1
[['0100.a2.txt', 171], ['0101.a2.txt', 167], ['0102.a2.txt', 153], ['0103.a2.txt', 163], ['0104.a2.txt', 168], ['0105.a2.txt', 135], ['0106.a2.txt', 145]]
0107.a2.txt 141 123 61 55 2 0 2
[['0100.a2.txt', 171], ['0101.a2.txt', 167], ['0102.a2.txt', 153], ['0103.a2.txt', 163], ['0104.a2.txt', 168], ['0105.a2.txt', 

In [13]:
print(doc) mina@cas.au.dk

<doc.id=0502.c1>
<title> How the understanding of Wuthering Heights is affected by the structure and the two narrators </title>
In this essay I will discuss the effect with the dual narrators in Wuthering Heights. I will also look at the structure and see what effect that has for the story. Does the book become more interesting with the two narrators, and does the structure contribute to make the story better or does the book become less interesting to read with this construction? First I will discuss the two narrators before I comment on the structure.
	 The main narrator is Mr. Lockwood but within his story Nelly is the narrator. I think Emily Bronte has chosen Mr. Lockwood as the main narrator to make the story more reliable. He is a man and he comes from the upper class while Nelly is a female servant. When the book was written it was more accepted with a male narrator than with a female, and I think that is why Bronte has chosen Mr. Lockwood as her principal narrator.
	 Another re

In [7]:
# create empty list
entities = []

# add each entity to list
for ent in doc.ents:
    entities.append(ent.text)
print(set(entities))

{'L. Cookson', 'Vogler', 'B. Longlerey\n\n', '43', 'Paul', 'Earnshaw', 'Thomas A.', 'Heights', 'Catherine', '27', 'Wuthering Heights', 'America', 'another half-hour', 'Bronte', 'Dean', 'Story and History in Wuthering Heights', '1802', '88', 'Norgate', 'chapter 31', 'Grange', 'Heatcliff', '83', 'eleven', '16', 'First', '248', 'Emily Bronte', 'Nelly', 'Longman Critical Essays, Eds', 'ruff', '139', 'Carl R.', 'The Narrators of Wuthering Heights', 'Woodring', '95', '1801', 'London', '1994', 'two', 'Lockwood', '257'}


In [8]:
# create counters for nouns, verbs, adjectives, and adverbs.
noun_count = 0
verb_count = 0
adj_count = 0
adv_count = 0

# with a for loop, add 1 to the counter every time that part of speech appears in the doc object
for token in doc:
    if token.pos_ == "NOUN":
        noun_count += 1
    elif token.pos_ == "VERB":
        verb_count += 1
    elif token.pos_ == "ADJ":
        adj_count += 1
    elif token.pos_ == "ADV":
        adv_count += 1

In [22]:
# calculating the relative frequency of each part of speech per 10,000 words
relative_freq_noun = (noun_count/len(doc)) * 10000
round(relative_freq_noun, 2)

1378.0

In [23]:
relative_freq_verb = (verb_count/len(doc)) * 10000
round(relative_freq_verb, 2)

1453.86

In [25]:
relative_freq_adj = (adj_count/len(doc)) * 10000
round(relative_freq_adj, 2)

518.33

In [26]:
relative_freq_adv = (adv_count/len(doc)) * 10000
round(relative_freq_adv, 2)

695.32

In [10]:
# extracting named entities for persons, locations, and organizations
persons = set()
for ent in doc.ents:
        if ent.label_ == 'PERSON':
            persons.add(ent.text)

num_persons = len(persons)
print(num_persons)

15


In [29]:
locations = set()
for ent in doc.ents:
        if ent.label_ == 'LOC':
            locations.add(ent.text)

num_locations = len(locations)
print(num_locations)

0


In [30]:
organisations = set()
for ent in doc.ents:
        if ent.label_ == 'ORG':
            organisations.add(ent.text)

num_organisations = len(organisations)
print(num_organisations)

0


In [20]:
annotations = []
for token in doc:
    annotations.append((token.text, token.pos_, noun_count, verb_count, adj_count, adv_count))

In [18]:
# creating a dataframe with pandas using the spaCy doc
# make one per subfolder
data = pd.DataFrame(annotations, 
                    columns=["Filename", "RelFreq NOUN", "RelFreq VERB", "RelFreq ADJ", "RelFreq ADV", "Unique PER", "Unique LOC", "Unique ORG"])

ValueError: 8 columns passed, passed data had 3 columns

In [15]:
data

,RelFreq NOUNS,RelFreq VERB
0,<,X
1,doc.id=0502.c1,X
2,>,SYM
3,\n,SPACE
4,<,X
...,...,...
1573,"""",PUNCT
1574,\n,SPACE
1575,<,X
1576,/doc,PUNCT


In [ ]:
# upload dataframe to output folder
outpath = os.path.join("..", "output", "annotations.csv")
data.to_csv(outpath)

issues:
- weird tokens? how to preprocess
- takes the last file instead of going through all of them - need to somehow append every file's text to the existing doc object
- cant make data frame with more than two columns
- add in the rest of the tasks to the loop